In [48]:
import pandas as pd
from collections import defaultdict, deque
import networkx as nx   
import matplotlib.pyplot as plt

## Load dữ liệu

In [49]:
nodes_df = pd.read_csv("/kaggle/input/network/nodes.csv")
edges_df = pd.read_csv("/kaggle/input/network/edges.csv")

print("Số lượng node:", len(nodes_df))
print("Số lượng cạnh:", len(edges_df))

nodes_df.head(), edges_df.head()


Số lượng node: 3939
Số lượng cạnh: 10122


(                                                link  \
 0       /wiki/Gi%E1%BA%A3i_Nobel_V%E1%BA%ADt_l%C3%BD   
 1      /wiki/Gi%E1%BA%A3i_Nobel_H%C3%B3a_h%E1%BB%8Dc   
 2  /wiki/Gi%E1%BA%A3i_Nobel_Sinh_l%C3%BD_h%E1%BB%...   
 3      /wiki/Gi%E1%BA%A3i_Nobel_V%C4%83n_h%E1%BB%8Dc   
 4        /wiki/Gi%E1%BA%A3i_Nobel_H%C3%B2a_b%C3%ACnh   
 
                                 name   type  
 0                  Giải Nobel Vật lý  EVENT  
 1                 Giải Nobel hóa học  EVENT  
 2  Giải Nobel Sinh lý học hoặc Y học  EVENT  
 3                 Giải Nobel Văn học  EVENT  
 4                Giải Nobel Hòa bình  EVENT  ,
                             src  \
 0          /wiki/George_Stigler   
 1  /wiki/Martinus_J._G._Veltman   
 2       /wiki/Tjalling_Koopmans   
 3          /wiki/Robert_Mundell   
 4       /wiki/Christian_de_Duve   
 
                                                  des  \
 0           /wiki/%C4%90%E1%BA%A1i_h%E1%BB%8Dc_Brown   
 1  /w/index.php?title=Peter_van_Nieuwenh

In [50]:
link_to_name = {}
name_to_link = {}

In [51]:
for _, row in nodes_df.iterrows():
    link = str(row['link']).strip()
    name = str(row['name']).strip()
    type_ = str(row['type']).strip()
    
    link_to_name[link] = name
    # Một tên có thể xuất hiện nhiều lần → lấy cái đầu tiên
    if name not in name_to_link:
        name_to_link[name] = link

## 3. Xây dựng đồ thị NetworkX

In [52]:
graph = defaultdict(list)

for _, row in edges_df.iterrows():
    src = str(row['src']).strip()
    des = str(row['des']).strip()
    
    if src in link_to_name and des in link_to_name:
        graph[src].append(des)

In [53]:
def shortest_path(start_link: str, goal_link: str):
    if start_link not in link_to_name:
        print(f"Không tìm thấy node khởi đầu: {start_link}")
        return None
    if goal_link not in link_to_name:
        print(f"Không tìm thấy node đích: {goal_link}")
        return None

    queue = deque([(start_link, [start_link])])
    visited = {start_link}

    while queue:
        current, path = queue.popleft()

        if current == goal_link:
            named_path = [link_to_name[link] for link in path]
            return named_path, path

        for neighbor in graph[current]:
            if neighbor not in visited:
                visited.add(neighbor)
                queue.append((neighbor, path + [neighbor]))

    return None 

## HÀM TIỆN ÍCH TÌM THEO TÊN

In [54]:

def find_path(start_name: str, goal_name: str):
    start_link = name_to_link.get(start_name)
    goal_link = name_to_link.get(goal_name)

    if not start_link:
        candidates = [name for name in name_to_link.keys() if start_name.lower() in name.lower()]
        if candidates:
            print(f"Không tìm thấy chính xác '{start_name}'. Bạn có ý muốn nói:")
            for c in candidates[:5]:
                print(f"  - {c}")
            if len(candidates) > 5:
                print(f"  ... và {len(candidates)-5} cái khác")
        else:
            print(f"Không tìm thấy node nào gần giống '{start_name}'")
        return

    if not goal_link:
        candidates = [name for name in name_to_link.keys() if goal_name.lower() in name.lower()]
        if candidates:
            print(f"Không tìm thấy chính xác '{goal_name}'. Bạn có ý muốn nói:")
            for c in candidates[:5]:
                print(f"  - {c}")
        else:
            print(f"Không tìm thấy node nào gần giống '{goal_name}'")
        return

    result = shortest_path(start_link, goal_link)
    if result:
        named_path, link_path = result
        print(f"Đường đi ngắn nhất ({len(named_path)-1} cạnh):")
        for i, name in enumerate(named_path, 1):
            print(f"  {i}. {name}")
        print("\nChi tiết quan hệ:")
        for i in range(len(link_path)-1):
            src = link_path[i]
            des = link_path[i+1]
            # Tìm loại quan hệ (nếu có trong edges.csv)
            rel = edges_df[(edges_df['src'] == src) & (edges_df['des'] == des)]['type']
            rel_type = rel.iloc[0] if not rel.empty else "liên kết"
            print(f"    → [{link_to_name[src]}] --({rel_type})--> [{link_to_name[des]}]")
    else:
        print("Không tồn tại đường đi giữa hai node này trong đồ thị hiện tại :(")

## 5. THUẬT TOÁN SHORT PATH

In [ ]:
print("TÌM ĐƯỜNG NGẮN NHẤT \n")

# Các ví dụ chạy tốt 100%
find_path("Giải Nobel Văn học", "Bob Dylan")
print("\n" + "="*70)

find_path("Albert Einstein", "Niels Bohr")
print("\n" + "="*70)

find_path("Marie Curie", "Irène Joliot-Curie")
print("\n" + "="*70)

find_path("Giải Nobel Vật lý", "Richard Feynman")
print("\n" + "="*70)


find_path("Giải Nobel Văn chương", "Bob Dylan")  


print("\nNhập 'q' để thoát")
while True:
    try:
        a = input("\nTừ node: ").strip()
        if a.lower() in ['q', 'quit', 'exit', '']:
            break
        b = input("Đến node: ").strip()
        if b.lower() in ['q', 'quit', 'exit', '']:
            break
        find_path(a, b)
    except KeyboardInterrupt:
        print("\nTạm biệt!")
        break

TÌM ĐƯỜNG NGẮN NHẤT 

Không tồn tại đường đi giữa hai node này trong đồ thị hiện tại :(

Không tồn tại đường đi giữa hai node này trong đồ thị hiện tại :(

Đường đi ngắn nhất (1 cạnh):
  1. Marie Curie
  2. Irène Joliot-Curie

Chi tiết quan hệ:
    → [Marie Curie] --(Các nghiên cứu sinh nổi tiếng)--> [Irène Joliot-Curie]

Không tồn tại đường đi giữa hai node này trong đồ thị hiện tại :(

Không tìm thấy node nào gần giống 'Giải Nobel Văn chương'

Nhập 'q' để thoát



Từ node:  Marie Curie
Đến node:  Irène Joliot-Curie


Đường đi ngắn nhất (1 cạnh):
  1. Marie Curie
  2. Irène Joliot-Curie

Chi tiết quan hệ:
    → [Marie Curie] --(Các nghiên cứu sinh nổi tiếng)--> [Irène Joliot-Curie]



Từ node:  Walther Nernst
Đến node:  Alber Einstein


Không tìm thấy node nào gần giống 'Alber Einstein'



Từ node:  Alber Einstein
Đến node:  Mileva Maric


Không tìm thấy node nào gần giống 'Alber Einstein'



Từ node:  Paul Samuelson
Đến node:  Robert Solow


Đường đi ngắn nhất (1 cạnh):
  1. Paul Samuelson
  2. Robert Solow

Chi tiết quan hệ:
    → [Paul Samuelson] --(Ảnh hưởng tới)--> [Robert Solow]
